<a href="https://colab.research.google.com/github/tennille-bernard/Kal-Academy-Assignments/blob/main/Assignment_8_TB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Important Note**  
The original link (https://www.kaggle.com/code/andrewmvd/covid-19-ct-scans-getting-started/input) did not provide a full download of the data. A search in Kaggle took me here (https://www.kaggle.com/datasets/andrewmvd/covid19-ct-scans) to what looks like identical data.

In [ ]:
#importing relevant packages to set up the json file and determine filepath
import os
import pandas as pd
from zipfile import ZipFile
!mkdir ~p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘~p’: File exists
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
#downloading the dataset
!kaggle datasets download -d andrewmvd/covid19-ct-scans

#Unzip the dataset to view the files
#!unzip covid19-ct-scans.zip -d covid19-ct-scans
with ZipFile('covid19-ct-scans.zip', 'r') as zip_ref:
    zip_ref.extractall('covid19-ct-scans')

Dataset URL: https://www.kaggle.com/datasets/andrewmvd/covid19-ct-scans
License(s): other


**As the files are in NifTi format (.nii)**  
it is recommended that we use the ***nibabel*** package. We have also been informed the imaged were rotated in the source, and should use a np.rot90 to augment for this.

**Importing the remaining packages**  


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile
import cv2
from skimage import io
import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from IPython.display import display
from tensorflow.keras import backend as K
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import glob
import random
from google.colab import files #library to upload files to colab notebook
%matplotlib inline

import plotly.graph_objects as go

import random
import os
from skimage import io
from PIL  import Image

import glob
import nibabel as nib

In [ ]:
def read_nii(filepath):
    '''
    Reads .nii file and returns pixel array
    '''
    ct_scan = nib.load(filepath)
    array   = ct_scan.get_fdata()
    array   = np.rot90(np.array(array))
    return(array)

**Loading the data containing the path to Covid19 scans and the corresponding mask**

In [ ]:
covid19_df = pd.read_csv('covid19-ct-scans/metadata.csv')
covid19_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ct_scan                  20 non-null     object
 1   lung_mask                20 non-null     object
 2   infection_mask           20 non-null     object
 3   lung_and_infection_mask  20 non-null     object
dtypes: object(4)
memory usage: 772.0+ bytes


In [ ]:
covid19_df.head()

,ct_scan,lung_mask,infection_mask,lung_and_infection_mask
0,../input/covid19-ct-scans/ct_scans/coronacases...,../input/covid19-ct-scans/lung_mask/coronacase...,../input/covid19-ct-scans/infection_mask/coron...,../input/covid19-ct-scans/lung_and_infection_m...
1,../input/covid19-ct-scans/ct_scans/coronacases...,../input/covid19-ct-scans/lung_mask/coronacase...,../input/covid19-ct-scans/infection_mask/coron...,../input/covid19-ct-scans/lung_and_infection_m...
2,../input/covid19-ct-scans/ct_scans/coronacases...,../input/covid19-ct-scans/lung_mask/coronacase...,../input/covid19-ct-scans/infection_mask/coron...,../input/covid19-ct-scans/lung_and_infection_m...
3,../input/covid19-ct-scans/ct_scans/coronacases...,../input/covid19-ct-scans/lung_mask/coronacase...,../input/covid19-ct-scans/infection_mask/coron...,../input/covid19-ct-scans/lung_and_infection_m...
4,../input/covid19-ct-scans/ct_scans/coronacases...,../input/covid19-ct-scans/lung_mask/coronacase...,../input/covid19-ct-scans/infection_mask/coron...,../input/covid19-ct-scans/lung_and_infection_m...


In [ ]:
#Update the file paths in the DataFrame

# Update the file paths in the DataFrame
base_dir = 'covid19-ct-scans/'
covid19_df['ct_scan'] = covid19_df['ct_scan'].apply(lambda x: os.path.join(base_dir, x.split('/')[-2], x.split('/')[-1]))
covid19_df['lung_mask'] = covid19_df['lung_mask'].apply(lambda x: os.path.join(base_dir, x.split('/')[-2], x.split('/')[-1]))
covid19_df['infection_mask'] = covid19_df['infection_mask'].apply(lambda x: os.path.join(base_dir, x.split('/')[-2], x.split('/')[-1]))
covid19_df['lung_and_infection_mask'] = covid19_df['lung_and_infection_mask'].apply(lambda x: os.path.join(base_dir, x.split('/')[-2], x.split('/')[-1]))

**Prepare Data for ImageDataGenerator**

In [ ]:
# Create a directory to store the processed data
processed_dir = 'processed_data'
image_dir = os.path.join(processed_dir, 'images')
mask_dir = os.path.join(processed_dir, 'masks')
os.makedirs(image_dir, exist_ok=True)
os.makedirs(mask_dir, exist_ok=True)

# Create a new DataFrame for the generator
generator_df = pd.DataFrame(columns=['image_path', 'label_path'])

from skimage.transform import resize
#IMG_SIZE_SAVE = (64, 64) # Consistent size for saving
IMG_SIZE_SAVE = (128, 128) # Consistent size for saving

# Process each row, save resized .npy files (SLICE BY SLICE)
for index, row in covid19_df.iterrows():
    try:
        ct_scan = read_nii(row['ct_scan'])
        lung_mask = read_nii(row['lung_mask'])
        infection_mask = read_nii(row['infection_mask'])

        # Iterate through each slice (assuming the last dimension is the slice)
        for slice_index in range(ct_scan.shape[-1]):
            ct_slice = ct_scan[:, :, slice_index]
            lung_slice = lung_mask[:, :, slice_index]
            infection_slice = infection_mask[:, :, slice_index]

            categorical_slice = np.zeros_like(lung_slice, dtype=np.uint8)
            categorical_slice[lung_slice > 0] = 1
            categorical_slice[infection_slice > 0] = 2

            # Resize slices
            resized_ct_slice = resize(ct_slice, IMG_SIZE_SAVE, anti_aliasing=True).astype(np.float32)
            resized_label_slice = resize(categorical_slice, IMG_SIZE_SAVE, order=0, preserve_range=True, anti_aliasing=False).astype(np.uint8)

            image_filename = f'image_{index}_slice_{slice_index}.npy'
            label_filename = f'label_{index}_slice_{slice_index}.npy'
            image_path = os.path.join(image_dir, image_filename)
            label_path = os.path.join(mask_dir, label_filename)
            np.save(image_path, resized_ct_slice.astype(np.float32))
            np.save(label_path, resized_label_slice)

            print(f"Saved slice {slice_index} - image shape: {resized_ct_slice.shape}, label shape: {resized_label_slice.shape}")

    except Exception as e:
        print(f"Error processing row {index}: {e}")


Saved slice 0 - image shape: (128, 128), label shape: (128, 128)
Saved slice 1 - image shape: (128, 128), label shape: (128, 128)
Saved slice 2 - image shape: (128, 128), label shape: (128, 128)
Saved slice 3 - image shape: (128, 128), label shape: (128, 128)
Saved slice 4 - image shape: (128, 128), label shape: (128, 128)
Saved slice 5 - image shape: (128, 128), label shape: (128, 128)
Saved slice 6 - image shape: (128, 128), label shape: (128, 128)
Saved slice 7 - image shape: (128, 128), label shape: (128, 128)
Saved slice 8 - image shape: (128, 128), label shape: (128, 128)
Saved slice 9 - image shape: (128, 128), label shape: (128, 128)
Saved slice 10 - image shape: (128, 128), label shape: (128, 128)
Saved slice 11 - image shape: (128, 128), label shape: (128, 128)
Saved slice 12 - image shape: (128, 128), label shape: (128, 128)
Saved slice 13 - image shape: (128, 128), label shape: (128, 128)
Saved slice 14 - image shape: (128, 128), label shape: (128, 128)
Saved slice 15 - ima

In [ ]:
# --- CHECK FILES IN DIRECTORIES ---
print("\nFiles in image directory:")
print(os.listdir(image_dir))
print("\nFiles in mask directory:")
print(os.listdir(mask_dir))


Files in image directory:
['image_2_slice_17.npy', 'image_0_slice_258.npy', 'image_5_slice_46.npy', 'image_9_slice_203.npy', 'image_9_slice_105.npy', 'image_11_slice_11.npy', 'image_8_slice_29.npy', 'image_19_slice_0.npy', 'image_0_slice_115.npy', 'image_11_slice_268.npy', 'image_11_slice_241.npy', 'image_19_slice_13.npy', 'image_0_slice_253.npy', 'image_0_slice_219.npy', 'image_11_slice_139.npy', 'image_12_slice_57.npy', 'image_5_slice_165.npy', 'image_3_slice_187.npy', 'image_4_slice_169.npy', 'image_1_slice_39.npy', 'image_6_slice_137.npy', 'image_9_slice_229.npy', 'image_2_slice_1.npy', 'image_0_slice_121.npy', 'image_11_slice_1.npy', 'image_6_slice_76.npy', 'image_6_slice_136.npy', 'image_8_slice_10.npy', 'image_11_slice_387.npy', 'image_19_slice_12.npy', 'image_4_slice_16.npy', 'image_11_slice_397.npy', 'image_7_slice_53.npy', 'image_4_slice_113.npy', 'image_11_slice_234.npy', 'image_2_slice_42.npy', 'image_0_slice_71.npy', 'image_1_slice_129.npy', 'image_11_slice_206.npy', 'ima

In [ ]:
# --- INSPECT SHAPE OF SAVED .NPY FILES ---
import random
num_files_to_inspect = min(5, len(os.listdir(image_dir)))
image_files_to_inspect = random.sample(os.listdir(image_dir), num_files_to_inspect)
print("\nInspecting shapes of a few saved image files:")
for filename in image_files_to_inspect:
    filepath = os.path.join(image_dir, filename)
    loaded_array = np.load(filepath)
    print(f"{filename}: {loaded_array.shape}")

num_files_to_inspect = min(5, len(os.listdir(mask_dir)))
mask_files_to_inspect = random.sample(os.listdir(mask_dir), num_files_to_inspect)
print("\nInspecting shapes of a few saved mask files:")
for filename in mask_files_to_inspect:
    filepath = os.path.join(mask_dir, filename)
    loaded_array = np.load(filepath)
    print(f"{filename}: {loaded_array.shape}")



Inspecting shapes of a few saved image files:
image_3_slice_156.npy: (128, 128)
image_4_slice_107.npy: (128, 128)
image_6_slice_171.npy: (128, 128)
image_1_slice_108.npy: (128, 128)
image_3_slice_186.npy: (128, 128)

Inspecting shapes of a few saved mask files:
label_4_slice_212.npy: (128, 128)
label_4_slice_80.npy: (128, 128)
label_7_slice_219.npy: (128, 128)
label_11_slice_130.npy: (128, 128)
label_11_slice_133.npy: (128, 128)


In [ ]:
# Define directories and parameters
#image_dir = 'processed_data/images'
#mask_dir = 'processed_data/masks'
IMG_SIZE = (128, 128)
BATCH_SIZE = 10
NUM_CLASSES = 3

In [ ]:
# Get lists of all .npy files in the image and mask directories
image_files = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.npy')])
mask_files = sorted([os.path.join(mask_dir, f) for f in os.listdir(mask_dir) if f.endswith('.npy')])

In [ ]:
# Ensure the number of image and mask files matches
assert len(image_files) == len(mask_files), "Number of image and mask files do not match!"


In [ ]:
'''def load_and_preprocess_image(file_path):
    image_bytes = tf.io.read_file(file_path)
    image = tf.io.decode_raw(image_bytes, tf.float64)
    image = tf.reshape(image, IMG_SIZE + (1,))
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image '''

'''# last stable version
def load_npy_image(file_path_tensor):
    def load_and_return(file_path):
        loaded_array = np.load(file_path.decode('utf-8')).astype(np.float32)
        print(f"Loading image: {file_path.decode('utf-8')}, Shape: {loaded_array.shape}")
        return loaded_array

    image = tf.numpy_function(
        func=load_and_return,
        inp=[file_path_tensor],
        Tout=tf.float32
    )
    image = tf.reshape(image, IMG_SIZE + (1,))
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image'''

def load_npy_image(file_path_tensor):
    def load_and_return(file_path):
        loaded_array = np.load(file_path.decode('utf-8')).astype(np.float32)
        return loaded_array
    image = tf.numpy_function(load_and_return, [file_path_tensor], tf.float32)
    image = tf.reshape(image, IMG_SIZE + (1,))
    # Convert grayscale to RGB by repeating the channel
    image = tf.image.grayscale_to_rgb(image)
    image = tf.image.resize(image, IMG_SIZE) # Ensure it's still IMG_SIZE after conversion
    print(f"Shape of image after loading and conversion: {image.shape}")
    return image

In [ ]:
'''def load_and_preprocess_mask(file_path):
    mask_bytes = tf.io.read_file(file_path)
    mask = tf.io.decode_raw(mask_bytes, tf.uint8)
    mask = tf.reshape(mask, IMG_SIZE)
    mask = tf.one_hot(mask, depth=NUM_CLASSES)
    return mask'''

def load_npy_mask(file_path_tensor):
    def load_and_return(file_path):
        loaded_array = np.load(file_path.decode('utf-8')).astype(np.uint8)
        print(f"Loading mask: {file_path.decode('utf-8')}, Shape: {loaded_array.shape}")
        return loaded_array

    mask = tf.numpy_function(
        func=load_and_return,
        inp=[file_path_tensor],
        Tout=tf.uint8
    )
    mask = tf.reshape(mask, IMG_SIZE)
    mask = tf.one_hot(mask, depth=NUM_CLASSES)
    return mask

**DEBUGGING:** Checking Tensor version

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.18.0


In [ ]:
# Create a dataset of file paths
image_file_ds = tf.data.Dataset.from_tensor_slices(image_files)
mask_file_ds = tf.data.Dataset.from_tensor_slices(mask_files)

# Load and preprocess the data using .map()
image_ds = image_file_ds.map(load_npy_image)
mask_ds = mask_file_ds.map(load_npy_mask)

# Combine the image and mask datasets
dataset = tf.data.Dataset.zip((image_ds, mask_ds))

# Convert to NumPy arrays for splitting
image_arrays = np.array(list(image_ds.as_numpy_iterator()))
mask_arrays = np.array(list(mask_ds.as_numpy_iterator()))


Shape of image after loading and conversion: (128, 128, 3)
Loading mask: processed_data/masks/label_0_slice_0.npy, Shape: (128, 128)
Loading mask: processed_data/masks/label_0_slice_1.npy, Shape: (128, 128)
Loading mask: processed_data/masks/label_0_slice_10.npy, Shape: (128, 128)
Loading mask: processed_data/masks/label_0_slice_100.npy, Shape: (128, 128)
Loading mask: processed_data/masks/label_0_slice_101.npy, Shape: (128, 128)
Loading mask: processed_data/masks/label_0_slice_102.npy, Shape: (128, 128)
Loading mask: processed_data/masks/label_0_slice_103.npy, Shape: (128, 128)
Loading mask: processed_data/masks/label_0_slice_104.npy, Shape: (128, 128)
Loading mask: processed_data/masks/label_0_slice_105.npy, Shape: (128, 128)
Loading mask: processed_data/masks/label_0_slice_106.npy, Shape: (128, 128)
Loading mask: processed_data/masks/label_0_slice_107.npy, Shape: (128, 128)
Loading mask: processed_data/masks/label_0_slice_108.npy, Shape: (128, 128)
Loading mask: processed_data/masks

**Train/Test Split**

In [ ]:
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, AveragePooling2D # Import necessary layers
from tensorflow.keras.models import Model # Import the Model class
from tensorflow.keras.applications import ResNet50 # Import ResNet50
from tensorflow.keras.callbacks import EarlyStopping # Import EarlyStopping


# Define input shape for a single 2D slice
IMG_SIZE = (128, 128)
#IMG_SIZE = (64,64)
input_shape = (IMG_SIZE[0], IMG_SIZE[1], 3)


In [ ]:
#train_df, test_df = train_test_split(generator_df, test_size=0.15, random_state=42) # Add random_state for reproducibility
# Split the arrays
X_train, X_test, y_train, y_test = train_test_split(image_arrays, mask_arrays, test_size=0.15, random_state=42)


In [ ]:
# Create TensorFlow Datasets for training and testing
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(BATCH_SIZE).shuffle(buffer_size=len(X_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE)

print(f"Number of training samples: {len(X_train)}")
print(f"Number of testing samples: {len(X_test)}")

Number of training samples: 2992
Number of testing samples: 528


In [ ]:
#Using ResNet50 as the Base Model
#from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input #move to cell with import packages

In [ ]:
# Define input shape for a single 2D slice
input_shape = (IMG_SIZE[0], IMG_SIZE[1], 3)
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=input_shape))
base_model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 134, 134,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 64, 64,    │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 64, 64,    │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 64, 64,    │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 66, 66,    │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 32, 32,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 32, 32,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 32, 32,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 32, 32,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 32, 32,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 32, 32,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 32, 32,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 32, 32,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 32, 32,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 32, 32,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 32, 32,    │      1,024 │ conv2_block1_3_c

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 23,534,592 (89.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [ ]:
#Freezing the model weights
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
'''# Add a classification head to the base model
head_model = base_model.output
head_model = AveragePooling2D(pool_size=(4, 4))(head_model)  # Adjust pool size if needed
head_model = Flatten(name='flatten')(head_model)
head_model = Dense(256, activation="relu")(head_model)
head_model = Dropout(0.3)(head_model)
head_model = Dense(256, activation="relu")(head_model)
head_model = Dropout(0.3)(head_model)
head_model = Dense(3, activation='softmax')(head_model)  # Assuming 3 classes (background, lung, infection)'''


'# Add a classification head to the base model\nhead_model = base_model.output\nhead_model = AveragePooling2D(pool_size=(4, 4))(head_model)  # Adjust pool size if needed\nhead_model = Flatten(name=\'flatten\')(head_model)\nhead_model = Dense(256, activation="relu")(head_model)\nhead_model = Dropout(0.3)(head_model)\nhead_model = Dense(256, activation="relu")(head_model)\nhead_model = Dropout(0.3)(head_model)\nhead_model = Dense(3, activation=\'softmax\')(head_model)  # Assuming 3 classes (background, lung, infection)'

In [ ]:
'''# Create the final model
model = Model(inputs=base_model.input, outputs=head_model)'''


'# Create the final model\nmodel = Model(inputs=base_model.input, outputs=head_model)'

In [ ]:
'''# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)'''

"# Compile the model\nmodel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])\n\n# Define EarlyStopping callback\nearly_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)"

In [ ]:
'''history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=100,  # Adjust as needed
    callbacks=[early_stopping])'''

'history = model.fit(\n    train_dataset,\n    validation_data=test_dataset,\n    epochs=100,  # Adjust as needed\n    callbacks=[early_stopping])'

In [ ]:
'''#Saving the model as a json file
model_json = model.to_json()
with open("model_json.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model_weights.h5")
print("Model saved to JSON and weights saved to H5")'''

'#Saving the model as a json file\nmodel_json = model.to_json()\nwith open("model_json.json", "w") as json_file:\n    json_file.write(model_json)\nmodel.save_weights("model_weights.h5")\nprint("Model saved to JSON and weights saved to H5")'

**Test the prediction and check model accuracy**

In [ ]:
'''# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_dataset, verbose=0)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")'''

'# Evaluate the model on the test set\nloss, accuracy = model.evaluate(test_dataset, verbose=0)\nprint(f"Test Loss: {loss:.4f}")\nprint(f"Test Accuracy: {accuracy:.4f}")'

In [ ]:
'''#Get predictions
predictions = model.predict(test_dataset)'''

'#Get predictions\npredictions = model.predict(test_dataset)'

In [ ]:
#use ResUNet for segmentation
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, Conv2DTranspose, concatenate, Input
from tensorflow.keras.models import Model

In [ ]:
def conv_block(input_tensor, num_filters):
    x = Conv2D(num_filters, 3, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

In [ ]:
def decoder_block(input_tensor, skip_tensor, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
    x = concatenate([x, skip_tensor])
    x = conv_block(x, num_filters)
    return x

In [ ]:
# Define input shape for a single 2D slice
input_shape = (IMG_SIZE[0], IMG_SIZE[1], 3)
resunet_base = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=input_shape))

In [ ]:
# Freeze the ResNet backbone
for layer in resunet_base.layers:
    layer.trainable = False

In [ ]:
def build_resunet(input_shape, num_classes, backbone):
    s1 = backbone.get_layer('conv1_relu').output        # (None, 64, 64, 64)
    s2 = backbone.get_layer('conv2_block3_out').output  # (None, 32, 32, 256)
    s3 = backbone.get_layer('conv3_block4_out').output  # (None, 16, 16, 512)
    s4 = backbone.get_layer('conv4_block6_out').output  # (None, 8, 8, 1024)

    b1 = backbone.output              # (None, 4, 4, 2048)

    # Decoder
    d1 = decoder_block(b1, s4, 1024)   # Upsamples 4x4 to 8x8
    d2 = decoder_block(d1, s3, 512)    # Upsamples 8x8 to 16x16
    d3 = decoder_block(d2, s2, 256)    # Upsamples 16x16 to 32x32
    d4 = decoder_block(d3, s1, 128)    # Upsamples 32x32 to 64x64

    # One more upsampling to reach 128x128
    u1 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(d4)
    # You might not have a direct skip connection of 128x128 from the encoder
    # You can either:
    # 1. Apply conv_block to u1 directly
    # 2. Try to find an earlier layer in ResNet with 128x128 (less likely after initial conv)

    final_output = Conv2D(num_classes, 1, activation='softmax', padding='same')(u1) # Output is 128x128

    model = Model(inputs=backbone.input, outputs=final_output)
    return model

In [ ]:
base_model.summary()

Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 134, 134,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 64, 64,    │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 64, 64,    │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 64, 64,    │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 66, 66,    │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 32, 32,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 32, 32,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 32, 32,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 32, 32,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 32, 32,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 32, 32,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 32, 32,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 32, 32,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 32, 32,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 32, 32,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 32, 32,    │      1,024 │ conv2_block1_3_c

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
resunet_model = build_resunet(input_shape, NUM_CLASSES, resunet_base)
resunet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # You might want Dice coefficient or IoU for segmentation
resunet_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 134, 134,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 64, 64,    │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 64, 64,    │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 64, 64,    │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 66, 66,    │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 32, 32,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 32, 32,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 32, 32,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 32, 32,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 32, 32,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 32, 32,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 32, 32,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 32, 32,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 32, 32,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 32, 32,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 32, 32,    │      1,024 │ conv2_block1_3_c

 Total params: 72,310,531 (275.84 MB)

 Trainable params: 48,715,139 (185.83 MB)

 Non-trainable params: 23,595,392 (90.01 MB)

In [ ]:
# Train the ResUNet model
resunet_history = resunet_model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=100,
    callbacks=[EarlyStopping(monitor='val_loss', patience=20, verbose=1)]
)

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_175']
Received: inputs=Tensor(shape=(None, 128, 128, 3))
  warnings.warn(msg)


300/300 ━━━━━━━━━━━━━━━━━━━━ 114s 256ms/step - accuracy: 0.9534 - loss: 0.1309 - val_accuracy: 0.9866 - val_loss: 0.0387
Epoch 2/100
300/300 ━━━━━━━━━━━━━━━━━━━━ 85s 140ms/step - accuracy: 0.9893 - loss: 0.0315 - val_accuracy: 0.9899 - val_loss: 0.0297
Epoch 3/100
300/300 ━━━━━━━━━━━━━━━━━━━━ 44s 144ms/step - accuracy: 0.9915 - loss: 0.0231 - val_accuracy: 0.9922 - val_loss: 0.0208
Epoch 4/100
300/300 ━━━━━━━━━━━━━━━━━━━━ 45s 147ms/step - accuracy: 0.9930 - loss: 0.0186 - val_accuracy: 0.9905 - val_loss: 0.0265
Epoch 5/100
300/300 ━━━━━━━━━━━━━━━━━━━━ 81s 145ms/step - accuracy: 0.9935 - loss: 0.0171 - val_accuracy: 0.3788 - val_loss: 10.9478
Epoch 6/100
300/300 ━━━━━━━━━━━━━━━━━━━━ 45s 148ms/step - accuracy: 0.9917 - loss: 0.0225 - val_accuracy: 0.9925 - val_loss: 0.0189
Epoch 7/100
300/300 ━━━━━━━━━━━━━━━━━━━━ 45s 149ms/step - accuracy: 0.9936 - loss: 0.0166 - val_accuracy: 0.9936 - val_loss: 0.0174
Epoch 8/100
300/300 ━━━━━━━━━━━━━━━━━━━━ 81s 146ms/step - accuracy: 0.9942 - loss: 0.0

In [ ]:
# Evaluate the ResUNet model
resunet_loss, resunet_accuracy = resunet_model.evaluate(test_dataset, verbose=0)
print(f"ResUNet Test Loss: {resunet_loss:.4f}")
print(f"ResUNet Test Accuracy: {resunet_accuracy:.4f}") # Accuracy might not be the best metric for segmentation

# Make predictions with ResUNet
segmentation_predictions = resunet_model.predict(test_dataset)

ResUNet Test Loss: 0.0124
ResUNet Test Accuracy: 0.9951


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_175']
Received: inputs=Tensor(shape=(10, 128, 128, 3))
  warnings.warn(msg)


53/53 ━━━━━━━━━━━━━━━━━━━━ 14s 138ms/step
